In [58]:
import torch.nn as nn
import torch.nn.functional as F
import torch as t
from torch.autograd import Variable
import torch.optim as optim

In [27]:
class Net(nn.Module):
    def __init__(self):
        #nn.module子类函数必须在构造函数中执行父类的构造函数
        super(Net,self).__init__()
        ##等价于 nn.moduel.__init__()
        ##卷积层1:单通道 6:输出6通道 5:卷积核5*5
        self.conv1=nn.Conv2d(1,6,5)
        self.conv2=nn.Conv2d(6,16,5)
        #fc层 y=wx+b
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
        
    def forward(self,x):
        #卷积--激活--池化
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        #reshape -1:自适应
        x=x.view(x.size(0),-1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    


In [28]:
net=Net()
print(net)

params=list(net.parameters())
print(len(params))
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())
input=Variable(t.randn(1,1,32,32))
out=net(input)
out.size()
##参数清零 反向传播
net.zero_grad()
out.backward(Variable(t.ones(1,10)))#反向传播

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [56]:
##Losing Function
output=net(input)
target=Variable(t.arange(0,10)) #arange 包含末尾数字
criterion=nn.MSELoss()
target=target.type(t.FloatTensor)
loss=criterion(output,target)
# loss.backward()

In [57]:
##运行Backward
net.zero_grad() #梯度清零
print('反向传播前conv1.bias的梯度')
print(net.conv1.bias.grad)
loss.backward()
print('backward后conv1.bias的梯度')
print(net.conv1.bias.grad)
print(net.conv1.bias.grad_fn)

反向传播前conv1.bias的梯度
tensor([0., 0., 0., 0., 0., 0.])
backward后conv1.bias的梯度
tensor([ 0.0727, -0.0148, -0.0589,  0.0480, -0.0588, -0.0269])
None


In [60]:
##SGD更新优化方法

##手动实现
# learning_rate=0.01
# for f in net.parameters():
#     f.data.sub_(f.grad.data*learning_rate)
    
optimizer=optim.SGD(net.parameters(),lr=0.01)
##训练时梯度清零
optimizer.zero_grad()
#计算损失
output=net(input)
loss=criterion(output,target)
#backward
loss.backward()
#更新参数
optimizer.step()